# Right Left up down, keyframes

## Setup

In [3]:
import os
from pathlib import Path
import pickle

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support

from frame_generator_kf import FrameGenerator
from model import create_model, visualise_model

In [4]:
DATA_PATH = "data_right_left_up_down_1200"
NOTEBOOK_NAME = "2_2d_plus_1_rlud_kf"
RESULTS_PATH = DATA_PATH + "/" + NOTEBOOK_NAME

# PARAMS
# number of frames taken from each video
n_frames = 10
#  ????
batch_size = 8
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

assert(os.path.isdir(DATA_PATH ))

if not os.path.isdir(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

print(f"classes being compared {index_df['category'].unique()}")

classes being compared ['Pushing something from left to right'
 'Pushing something from right to left' 'Moving something up'
 'Moving something down']


## Preprocess video data

Load something something data tf.data.Dataset

In [5]:
subset_paths = {
    "test": Path(f'{DATA_PATH}/test'),
    "train": Path(f'{DATA_PATH}/train'),
    "val": Path(f'{DATA_PATH}/validation'),
}

output_signature = (
    tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
    tf.TensorSpec(shape = (), dtype = tf.int16)
)

train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        index_df=index_df,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['val'],
        n_frames=n_frames,
        index_df=index_df,
        training=False
    ),
    output_signature = output_signature
)
# Batch the data
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(
        subset_paths['test'],
        n_frames=n_frames,
        index_df=index_df,
        training=False
    ),
    output_signature = output_signature
)

print(type(test_ds))

# Batch the data
test_ds = test_ds.batch(batch_size)

print(type(test_ds))


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


## Model Creation

In [6]:
model = create_model(n_frames=n_frames, height=HEIGHT, width=WIDTH)

## Validate Model and Data

In [7]:
# testing tensor is setup correct
iter(train_ds)

## Build Model

In [8]:
frames, label = next(iter(train_ds))

In [9]:
model.build(frames)

## Visualise Model

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 224, 22  0           []                               
                                4, 3)]                                                            
                                                                                                  
 conv2_plus1d (Conv2Plus1D)     (None, 10, 224, 224  3152        ['input_1[0][0]']                
                                , 16)                                                             
                                                                                                  
 batch_normalization (BatchNorm  (None, 10, 224, 224  64         ['conv2_plus1d[0][0]']           
 alization)                     , 16)                                                         

In [11]:
visualise_model(
    model=model,
    file_path=DATA_PATH + "/model_summary.dot"
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


## Load the Model

Using BinaryCrossentropy as it is more effective for binary data

from_logits is false because final layer includes a sigmoid activation,

In [12]:
previous_runs = 2

In [13]:
previous_runs += 1

model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=[
                    'accuracy',
                    # tf.keras.metrics.Precision(),
                    # tf.keras.metrics.Recall()
                    # tf.keras.metrics.Precision(class_id=0, name='precision_neg'),
                    # tf.keras.metrics.Precision(class_id=1, name='precision_pos'),
                    # tf.keras.metrics.Recall(class_id=0, name='recall_neg'),
                    # tf.keras.metrics.Recall(class_id=1, name='recall_pos')
                ]
            )

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    RESULTS_PATH + '/model-runs-' + str(previous_runs) + '-cp-{epoch:02d}-{val_loss:.2f}.ckpt',
    save_best_only=True,  # Save only the best model based on a monitored metric (e.g., val_loss), will only replace saved value if it is better
    monitor='val_loss',
    mode='min',  # 'min' for loss, 'max' for accuracy
    save_weights_only=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
)

In [14]:
latest = tf.train.latest_checkpoint(RESULTS_PATH)

if latest is not None:
    print(f"loading model from weights: {latest}")
    model.load_weights(latest)

loading model from weights: data_right_left_up_down_1200/2_2d_plus_1_rlud_kf/model-runs-3-cp-07-0.48.ckpt


## Train the Model

In [15]:
previously_run_epochs = 0 + 32 + 5

history = model.fit(
        x=train_ds,
        epochs = 50 - previously_run_epochs,
        validation_data=val_ds,
        callbacks=[checkpoint_callback, early_stopping],
    )

Epoch 1/13
480/480 [==============================] - 508s 1s/step - loss: 0.3123 - accuracy: 0.8898 - val_loss: 0.4927 - val_accuracy: 0.8271
Epoch 2/13
480/480 [==============================] - 478s 990ms/step - loss: 0.2837 - accuracy: 0.9076 - val_loss: 0.5644 - val_accuracy: 0.7833
Epoch 3/13
480/480 [==============================] - 478s 988ms/step - loss: 0.2735 - accuracy: 0.9102 - val_loss: 0.5076 - val_accuracy: 0.8271
Epoch 4/13
480/480 [==============================] - 478s 989ms/step - loss: 0.2483 - accuracy: 0.9164 - val_loss: 0.5890 - val_accuracy: 0.7958
Epoch 5/13
480/480 [==============================] - 480s 994ms/step - loss: 0.2600 - accuracy: 0.9089 - val_loss: 0.5667 - val_accuracy: 0.7979
Epoch 6/13
480/480 [==============================] - 475s 983ms/step - loss: 0.2536 - accuracy: 0.9089 - val_loss: 0.5257 - val_accuracy: 0.7979
Epoch 7/13
480/480 [==============================] - 478s 989ms/step - loss: 0.2296 - accuracy: 0.9255 - val_loss: 0.4848 - va

## Analyse results

In [15]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

true_labels = []
predictions = []

# Iterate over the batched test dataset
for batch in test_ds:
    x, y = batch  # x is the batch of features, y is the batch of labels
    true_labels.extend(y.numpy())  # Store true labels
    preds = model.predict(x)  # Generate predictions for the batch
    preds = softmax(preds, axis=1)  # Apply softmax to convert logits to probabilities
    preds = np.argmax(preds, axis=1)  # Get the class with the highest probability
    predictions.extend(preds)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predictions = np.array(predictions)

1/1 [==============================] - 0s 258ms/step


In [17]:
fg = FrameGenerator(
    subset_paths['test'],
    n_frames=n_frames,
    index_df=index_df,
    training=False
)
class_id_value = {
    fg.class_ids_for_name[x]: x for x in fg.class_ids_for_name.keys()
 }

# Convert lists to numpy arrays if they aren't already
true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)

# Calculate precision, recall, and F1-score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average=None)

# Print accuracy and F1-scores for each class
print(f"Overall Accuracy: {accuracy}")
for i, (prec, rec, f1) in enumerate(zip(precision, recall, f1_score)):
    print(f"Class {class_id_value[i]}: Precision: {prec}, Recall: {rec}, F1 Score: {f1}")

Overall Accuracy: 0.8020833333333334
Class Pushing something from left to right: Precision: 0.8074074074074075, Recall: 0.9083333333333333, F1 Score: 0.8549019607843137
Class Pushing something from right to left: Precision: 0.9279279279279279, Recall: 0.8583333333333333, F1 Score: 0.8917748917748917
Class Moving something up: Precision: 0.8404255319148937, Recall: 0.6583333333333333, F1 Score: 0.7383177570093459
Class Moving something down: Precision: 0.6714285714285714, Recall: 0.7833333333333333, F1 Score: 0.7230769230769231
